# Market Basket Analysis using Instacart Online Grocery Dataset
## Which products will an Instacart consumer purchase again?

To showcase the concept of market basket analysis with the [Databricks Unified Analytics Platform](https://databricks.com/product/unified-analytics-platform), we will use the Instacart's [3 Million Instacart Orders, Open Sourced](https://www.instacart.com/datasets/grocery-shopping-2017) dataset.

> “The Instacart Online Grocery Shopping Dataset 2017”, Accessed from https://www.instacart.com/datasets/grocery-shopping-2017 on 01/17/2018. This anonymized dataset contains a sample of over 3 million grocery orders from more than 200,000 Instacart users.
For each user, we provide between 4 and 100 of their orders, with the sequence of products purchased in each order. We also provide the week and hour of day the order was placed, and a relative measure of time between orders.

Whether you shop from meticulously planned grocery lists or let whimsy guide your grazing, our unique food rituals define who we are. Instacart's grocery ordering and delivery app aims to make it easy to fill your refrigerator and pantry with your personal favorites and staples when you need them. After selecting products through the Instacart app, personal shoppers review your order and do the in-store shopping and delivery for you.


This notebook will show how you can predict which items a shopper will purchase whether they buy it again or recommend to try for the first time.
<img src="https://s3.us-east-2.amazonaws.com/databricks-dennylee/media/buy+it+again+or+recommend.png" width="1100"/>


*Source: [3 Million Instacart Orders, Open Sourced](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2)*

![](https://s3.us-east-2.amazonaws.com/databricks-dennylee/media/data-engineering-pipeline-3.png)

Data engineering pipelines are commonly comprised of these components:

* **Ingest Data**: Bringing in the data from your source systems; often involving ETL processes (though we will skip this step in this demo for brevity)
* **Explore Data**: Now that you have cleansed data, explore it so you can get some business insight
* **Train ML Model**: Execute FP-growth for frequent pattern mining
* **Review Association Rules**: Review the generated association rules

# Ingest Data

First, download the [3 Million Instacart Orders, Open Sourced](https://www.instacart.com/datasets/grocery-shopping-2017) and upload it to `dbfs`; for more information, refer to [Importing Data](https://docs.databricks.com/user-guide/importing-data.html).

The following `dbutils filesystem (fs)` query displays the six files:
* `Orders`: 3.4M rows, 206K users
* `Products`: 50K rows
* `Aisles`: 134 rows 
* `Departments`: 21 rows
* `order_products__SET`: 30M+ rows where `SET` is defined as:
  * `prior`: 3.2M previous orders
  * `train`: 131K orders for your training dataset
  
Reference: [The Instacart Online Grocery Shopping Dataset 2017 Data Descriptions](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b)

### Important
You will need to **edit** the locations (in the examples below, we're using `/mnt/bhavin/mba/instacard/csv`) to where you had uploaded your data.

In [0]:
%fs ls /mnt/bhavin/mba/instacart/csv

path,name,size
dbfs:/mnt/bhavin/mba/instacart/csv/aisles.csv,aisles.csv,2603
dbfs:/mnt/bhavin/mba/instacart/csv/departments.csv,departments.csv,270
dbfs:/mnt/bhavin/mba/instacart/csv/order_products__prior.csv,order_products__prior.csv,577550706
dbfs:/mnt/bhavin/mba/instacart/csv/order_products__train.csv,order_products__train.csv,24680147
dbfs:/mnt/bhavin/mba/instacart/csv/orders.csv,orders.csv,108968645
dbfs:/mnt/bhavin/mba/instacart/csv/products.csv,products.csv,2166953


In [0]:
%fs head dbfs:/mnt/bhavin/mba/instacart/csv/orders.csv

[Truncated to first 65536 bytes]
order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
2539329,1,prior,1,2,08,
2398795,1,prior,2,3,07,15.0
473747,1,prior,3,3,12,21.0
2254736,1,prior,4,4,07,29.0
431534,1,prior,5,4,15,28.0
3367565,1,prior,6,2,07,19.0
550135,1,prior,7,1,09,20.0
3108588,1,prior,8,1,14,14.0
2295261,1,prior,9,1,16,0.0
2550362,1,prior,10,4,08,30.0
1187899,1,train,11,4,08,14.0
2168274,2,prior,1,2,11,
1501582,2,prior,2,5,10,10.0
1901567,2,prior,3,1,10,3.0
738281,2,prior,4,2,10,8.0
1673511,2,prior,5,3,11,8.0
1199898,2,prior,6,2,09,13.0
3194192,2,prior,7,2,12,14.0
788338,2,prior,8,1,15,27.0
1718559,2,prior,9,2,09,8.0
1447487,2,prior,10,1,11,6.0
1402090,2,prior,11,1,10,30.0
3186735,2,prior,12,1,09,28.0
3268552,2,prior,13,4,11,30.0
839880,2,prior,14,3,10,13.0
1492625,2,train,15,1,11,30.0
1374495,3,prior,1,1,14,
444309,3,prior,2,3,19,9.0
3002854,3,prior,3,3,16,21.0
2037211,3,prior,4,2,18,20.0
2710558,3,prior,5,0,17,12.0
1972919,3,prior,6,0,16,7.0
1839752,3,prior,7,0,15,7.0
3225766,3,prior,8,0,17,7.0
3160850,3,prior,9,0,16,7.0
676467,3,prior,10,3,16,17.0
521107,3,prior,11,0,18,11.0
1402502,3,prior,12,1,15,15.0
2774568,3,test,13,5,15,11.0
3343014,4,prior,1,6,11,
2030307,4,prior,2,4,11,19.0
691089,4,prior,3,4,15,21.0
94891,4,prior,4,5,13,15.0
2557754,4,prior,5,5,13,0.0
329954,4,test,6,3,12,30.0
2717275,5,prior,1,3,12,
1909121,5,prior,2,0,16,11.0
2267326,5,prior,3,3,18,10.0
157374,5,prior,4,1,18,19.0
2196797,5,train,5,0,11,6.0
2086598,6,prior,1,5,18,
298250,6,prior,2,4,16,6.0
998866,6,prior,3,2,18,12.0
1528013,6,test,4,3,16,22.0
2565571,7,prior,1,3,09,
2402008,7,prior,2,1,19,30.0
121053,7,prior,3,0,18,30.0
1695742,7,prior,4,2,10,9.0
3321109,7,prior,5,5,18,3.0
1682003,7,prior,6,1,18,10.0
1014379,7,prior,7,1,15,7.0
776265,7,prior,8,2,12,8.0
2507052,7,prior,9,6,07,4.0
1396119,7,prior,10,1,20,9.0
565870,7,prior,11,0,15,6.0
3239072,7,prior,12,0,14,7.0
1175028,7,prior,13,5,15,5.0
3060817,7,prior,14,1,13,17.0
3120740,7,prior,15,3,16,2.0
2425840,7,prior,16,0,13,11.0
23391,7,prior,17,0,10,28.0
2984707,7,prior,18,0,09,7.0
369015,7,prior,19,3,12,3.0
2452257,7,prior,20,3,09,7.0
525192,7,train,21,2,11,6.0
600894,8,prior,1,6,00,
2979257,8,prior,2,1,08,30.0
2570360,8,prior,3,5,00,30.0
880375,8,train,4,1,14,10.0
280530,9,prior,1,1,17,
2762092,9,prior,2,0,14,6.0
1830137,9,prior,3,5,12,30.0
1094988,9,train,4,6,10,30.0
1224907,10,prior,1,2,14,
68288,10,prior,2,5,15,30.0
2115522,10,prior,3,3,19,12.0
83395,10,prior,4,3,15,14.0
1353310,10,prior,5,5,20,23.0
1822501,10,train,6,0,19,30.0
2411567,11,prior,1,0,11,
1170872,11,prior,2,5,11,12.0
2840752,11,prior,3,5,10,14.0
1611810,11,prior,4,5,13,30.0
2921164,11,prior,5,5,11,30.0
790927,11,prior,6,5,11,7.0
1468214,11,prior,7,5,09,30.0
1376945,11,test,8,6,11,8.0
1483446,12,prior,1,1,12,
1247527,12,prior,2,5,09,30.0
1916106,12,prior,3,5,08,14.0
1057378,12,prior,4,3,09,26.0
221248,12,prior,5,1,09,30.0
1356845,12,test,6,1,20,30.0
2618231,13,prior,1,6,12,
2560699,13,prior,2,0,11,8.0
2288946,13,prior,3,6,21,6.0
19256,13,prior,4,1,12,9.0
1378982,13,prior,5,0,13,6.0
2363981,13,prior,6,0,18,7.0
1906169,13,prior,7,0,08,7.0
1757094,13,prior,8,6,18,13.0
519471,13,prior,9,6,17,7.0
2298068,13,prior,10,6,22,7.0
1472646,13,prior,11,0,14,8.0
1789302,13,prior,12,6,13,6.0
1827621,13,train,13,0,21,8.0
2906263,14,prior,1,2,18,
3206747,14,prior,2,5,08,30.0
1366559,14,prior,3,0,14,2.0
1385564,14,prior,4,6,08,20.0
1691859,14,prior,5,5,08,30.0
2127409,14,prior,6,1,08,30.0
2505309,14,prior,7,5,10,30.0
3067644,14,prior,8,4,08,27.0
2109666,14,prior,9,5,09,30.0
1625325,14,prior,10,4,21,30.0
1335295,14,prior,11,1,05,25.0
2045336,14,prior,12,5,09,4.0
3394109,14,prior,13,5,05,7.0
2316178,14,train,14,2,19,11.0
2514324,15,prior,1,5,13,
3135826,15,prior,2,6,12,15.0
2564002,15,prior,3,6,12,7.0
714991,15,prior,4,1,11,16.0
3317979,15,prior,5,4,15,17.0
876095,15,prior,6,5,11,8.0
2685110,15,prior,7,1,11,17.0
3119091,15,prior,8,2,17,8.0
887727,15,prior,9,2,13,7.0
2570697,15,prior,10,2,10,21.0
2600170,15,prior,1

In [0]:
# Import Data
aisles = spark.read.csv("/mnt/bhavin/mba/instacart/csv/aisles.csv", header=True, inferSchema=True)
departments = spark.read.csv("/mnt/bhavin/mba/instacart/csv/departments.csv", header=True, inferSchema=True)
order_products_prior = spark.read.csv("/mnt/bhavin/mba/instacart/csv/order_products__prior.csv", header=True, inferSchema=True)
order_products_train = spark.read.csv("/mnt/bhavin/mba/instacart/csv/order_products__train.csv", header=True, inferSchema=True)
orders = spark.read.csv("/mnt/bhavin/mba/instacart/csv/orders.csv", header=True, inferSchema=True)
products = spark.read.csv("/mnt/bhavin/mba/instacart/csv/products.csv", header=True, inferSchema=True)

# Create Temporary Tables
aisles.createOrReplaceTempView("aisles")
departments.createOrReplaceTempView("departments")
order_products_prior.createOrReplaceTempView("order_products_prior")
order_products_train.createOrReplaceTempView("order_products_train")
orders.createOrReplaceTempView("orders")
products.createOrReplaceTempView("products")

# Exploratory Data Analysis

Explore your Instacart data using Spark SQL

In [0]:
%sql
select 
  count(order_id) as total_orders, 
  (case 
     when order_dow = '0' then 'Sunday'
     when order_dow = '1' then 'Monday'
     when order_dow = '2' then 'Tuesday'
     when order_dow = '3' then 'Wednesday'
     when order_dow = '4' then 'Thursday'
     when order_dow = '5' then 'Friday'
     when order_dow = '6' then 'Saturday'              
   end) as day_of_week 
  from orders  
 group by order_dow 
 order by total_orders desc

total_orders,day_of_week
600905,Sunday
587478,Monday
467260,Tuesday
453368,Friday
448761,Saturday
436972,Wednesday
426339,Thursday


In [0]:
%sql
select 
  count(order_id) as total_orders, 
  order_hour_of_day as hour 
  from orders 
 group by order_hour_of_day 
 order by order_hour_of_day

total_orders,hour
22758,0
12398,1
7539,2
5474,3
5527,4
9569,5
30529,6
91868,7
178201,8
257812,9


In [0]:
%sql
select countbydept.*
  from (
  -- from product table, let's count number of records per dept
  -- and then sort it by count (highest to lowest) 
  select department_id, count(1) as counter
    from products
   group by department_id
   order by counter asc 
  ) as maxcount
inner join (
  -- let's repeat the exercise, but this time let's join
  -- products and departments tables to get a full list of dept and 
  -- prod count
  select
    d.department_id,
    d.department,
    count(1) as products
    from departments d
      inner join products p
         on p.department_id = d.department_id
   group by d.department_id, d.department 
   order by products desc
  ) countbydept 
  -- combine the two queries's results by matching the product count
  on countbydept.products = maxcount.counter

department_id,department,products
11,personal care,6563
19,snacks,6264
13,pantry,5371
7,beverages,4365
1,frozen,4007
16,dairy eggs,3449
17,household,3084
15,canned goods,2092
9,dry goods pasta,1858
4,produce,1684


In [0]:
%sql
select count(opp.order_id) as orders, p.product_name as popular_product
  from order_products_prior opp, products p
 where p.product_id = opp.product_id 
 group by popular_product 
 order by orders desc 
 limit 10

orders,popular_product
472565,Banana
379450,Bag of Organic Bananas
264683,Organic Strawberries
241921,Organic Baby Spinach
213584,Organic Hass Avocado
176815,Organic Avocado
152657,Large Lemon
142951,Strawberries
140627,Limes
137905,Organic Whole Milk


In [0]:
%sql
select d.department, count(distinct p.product_id) as products
  from products p
    inner join departments d
      on d.department_id = p.department_id
 group by d.department
 order by products desc
 limit 10

department,products
personal care,6563
snacks,6264
pantry,5371
beverages,4365
frozen,4007
dairy eggs,3449
household,3084
canned goods,2092
dry goods pasta,1858
produce,1684


## Organize and View Shopping Basket

In [0]:
# Organize the data by shopping basket
from pyspark.sql.functions import collect_set, col, count
rawData = spark.sql("select p.product_name, o.order_id from products p inner join order_products_train o where o.product_id = p.product_id")
baskets = rawData.groupBy('order_id').agg(collect_set('product_name').alias('items'))
baskets.createOrReplaceTempView('baskets')

In [0]:
display(baskets)

order_id,items
1342,"List(Raw Shrimp, Seedless Cucumbers, Versatile Stain Remover, Organic Strawberries, Organic Mandarins, Chicken Apple Sausage, Pink Lady Apples, Bag of Organic Bananas)"
1591,"List(Cracked Wheat, Strawberry Rhubarb Yoghurt, Organic Bunny Fruit Snacks Berry Patch, Goodness Grapeness Organic Juice Drink, Honey Graham Snacks, Spinach, Granny Smith Apples, Oven Roasted Turkey Breast, Pure Vanilla Extract, Chewy 25% Low Sugar Chocolate Chip Granola, Banana, Original Turkey Burgers Smoke Flavor Added, Twisted Tropical Tango Organic Juice Drink, Navel Oranges, Lower Sugar Instant Oatmeal Variety, Ultra Thin Sliced Provolone Cheese, Natural Vanilla Ice Cream, Cinnamon Multigrain Cereal, Garlic, Goldfish Pretzel Baked Snack Crackers, Original Whole Grain Chips, Medium Scarlet Raspberries, Lemon Yogurt, Original Patties (100965) 12 Oz Breakfast, Nutty Bars, Strawberry Banana Smoothie, Green Machine Juice Smoothie, Coconut Dreams Cookies, Buttermilk Waffles, Uncured Genoa Salami, Organic Greek Whole Milk Blended Vanilla Bean Yogurt)"
4519,List(Beet Apple Carrot Lemon Ginger Organic Cold Pressed Juice Beverage)
4935,List(Vodka)
6357,"List(Globe Eggplant, Panko Bread Crumbs, Fresh Mozzarella Ball, Grated Parmesan, Gala Apples, Italian Pasta Sauce Basilico Tomato, Basil & Garlic, Organic Basil, Banana, Provolone)"
10362,"List(Organic Baby Spinach, Organic Spring Mix, Organic Leek, Slow Roasted Lightly Seasoned Chick'n, Organic Basil, Organic Shredded Mild Cheddar, Bag of Organic Bananas, Sliced Baby Bella Mushrooms, Organic Tapioca Flour, Organic Gala Apples, Lemons, Limes, Pitted Dates, Jalapeno Peppers, Original Tofurky Deli Slices, Organic Red Bell Pepper, Organic Shredded Carrots, Roma Tomato, Crinkle Cut French Fries, Large Greenhouse Tomato, Organic Pinto Beans, Organic Three Grain Tempeh, Organic Garnet Sweet Potato (Yam), Organic Coconut Milk, Organic Extra Firm Tofu, Ground Sausage Style Veggie Protein, Extra Virgin Olive Oil, Hass Avocados, Multigrain Tortilla Chips, The Ultimate Beefless Burger, Yellow Bell Pepper, Coconut Flour, Light Brown Sugar, Organic Harissa Seasoning, Crushed Garlic, Organic Whole Cashews)"
19204,"List(Reduced Fat Crackers, Dishwasher Cleaner, Peanut Powder, Disinfecting Wipes Lemon & Fresh Scent, Lemon Lime Thirst Quencher, Fat Free & Lower Sodium Chicken Broth, American Blend Salad, Cinnamon Cereal, Extra Nasal Strips, Reduced Fat Creamy Peanut Butter Spread, Mozzarella String Cheese Sticks, Light Low-Moisture Part Skim, Electrolyte Enhanced Water, Original Petroleum Jelly, High Efficiency Complete Dual Formula)"
29601,"List(Organic Red Onion, Small Batch Authentic Taqueria Tortilla Chips, Hummus, Hope, Original Recipe, Unsweetened Whole Milk Peach Greek Yogurt, Toasted Coconut Almondmilk Blend, Skillet Refried Red Beans Sautéed With Onion & Tomatillo, Almondmilk, Pure, Chocolate Protein, Organic Greek Lowfat Yogurt With Strawberries, Bag of Organic Bananas, California Orange Juice, Mini Whole Wheat Pita Bread, Coconut Almond Creamer Blend, Banana Chia Pod, Tomatillo Salsa, SALSA FRNTR CHPTL SALSA, Guacamole, Water)"
31035,"List(Organic Cripps Pink Apples, Organic Golden Delicious Apple, Organic Navel Orange, Bag of Organic Bananas)"
40011,"List(Organic Baby Spinach, Organic Blues Bread with Blue Cornmeal Crust, Sea Salt Macadamias, Natural Calm Magnesium Supplement Raspberry Lemon Flavor Powder, Chocolate Coconut Protein Bar, Sport Chocolate Mint Protein Bar)"


# Train ML Model 

To understand the frequency of items are associated with each other (e.g. peanut butter and jelly), we will use association rule mining for market basket analysis.  [Spark MLlib](http://spark.apache.org/docs/latest/mllib-guide.html) implements two algorithms related to frequency pattern mining (FPM): [FP-growth](https://spark.apache.org/docs/latest/mllib-frequent-pattern-mining.html#fp-growth) and [PrefixSpan](https://spark.apache.org/docs/latest/mllib-frequent-pattern-mining.html#prefixspan). The distinction is that FP-growth does not use order information in the itemsets, if any, while PrefixSpan is designed for sequential pattern mining where the itemsets are ordered. We will use FP-growth as the order information is not important for this use case.

> Note, we will be using the `Scala API` so we can configure `setMinConfidence`.

In [0]:
%scala
import org.apache.spark.ml.fpm.FPGrowth

// Extract out the items 
val baskets_ds = spark.sql("select items from baskets").as[Array[String]].toDF("items")

// Use FPGrowth
val fpgrowth = new FPGrowth().setItemsCol("items").setMinSupport(0.001).setMinConfidence(0)
val model = fpgrowth.fit(baskets_ds)

import org.apache.spark.ml.fpm.FPGrowth
baskets_ds: org.apache.spark.sql.DataFrame = [items: array<string>]
fpgrowth: org.apache.spark.ml.fpm.FPGrowth = fpgrowth_1bacd1c33f93
model: org.apache.spark.ml.fpm.FPGrowthModel = fpgrowth_1bacd1c33f93

In [0]:
%scala
// Display frequent itemsets
val mostPopularItemInABasket = model.freqItemsets
mostPopularItemInABasket.createOrReplaceTempView("mostPopularItemInABasket")

mostPopularItemInABasket: org.apache.spark.sql.DataFrame = [items: array<string>, freq: bigint]

In [0]:
%sql
select items, freq from mostPopularItemInABasket where size(items) > 2 order by freq desc limit 20

items,freq
"List(Organic Hass Avocado, Organic Strawberries, Bag of Organic Bananas)",506
"List(Organic Raspberries, Organic Strawberries, Bag of Organic Bananas)",452
"List(Organic Baby Spinach, Organic Strawberries, Bag of Organic Bananas)",413
"List(Organic Raspberries, Organic Hass Avocado, Bag of Organic Bananas)",355
"List(Organic Hass Avocado, Organic Baby Spinach, Bag of Organic Bananas)",353
"List(Organic Avocado, Organic Baby Spinach, Banana)",338
"List(Organic Avocado, Large Lemon, Banana)",311
"List(Limes, Large Lemon, Banana)",300
"List(Organic Cucumber, Organic Strawberries, Bag of Organic Bananas)",284
"List(Organic Baby Spinach, Organic Strawberries, Banana)",271


# Review Association Rules

In addition to `freqItemSets`, the `FP-growth` model also generates `association rules`.  For example, if a shopper purchases *peanut butter* , what is the likelihood that they will also purchase *jelly*.  For more information, a good reference is Susan Li's [A Gentle Introduction on Market Basket Analysis — Association Rules](https://towardsdatascience.com/a-gentle-introduction-on-market-basket-analysis-association-rules-fa4b986a40ce)

In [0]:
%scala
// Display generated association rules.
val ifThen = model.associationRules
ifThen.createOrReplaceTempView("ifThen")

ifThen: org.apache.spark.sql.DataFrame = [antecedent: array<string>, consequent: array<string> ... 1 more field]

In [0]:
%sql
select antecedent as `antecedent (if)`, consequent as `consequent (then)`, confidence from ifThen order by confidence desc limit 20

antecedent (if),consequent (then),confidence
"List(Organic Raspberries, Organic Hass Avocado, Organic Strawberries)",List(Bag of Organic Bananas),0.6038461538461538
"List(Organic Kiwi, Organic Hass Avocado)",List(Bag of Organic Bananas),0.5665236051502146
"List(Organic Hass Avocado, Organic Baby Spinach, Organic Strawberries)",List(Bag of Organic Bananas),0.5470085470085471
"List(Organic Whole String Cheese, Organic Hass Avocado)",List(Bag of Organic Bananas),0.5422885572139303
"List(Yellow Onions, Strawberries)",List(Banana),0.5306122448979592
"List(Organic Navel Orange, Organic Raspberries)",List(Bag of Organic Bananas),0.5284974093264249
"List(Organic Navel Orange, Organic Hass Avocado)",List(Bag of Organic Bananas),0.5283018867924528
"List(Organic Cucumber, Organic Hass Avocado, Organic Strawberries)",List(Bag of Organic Bananas),0.5274725274725275
"List(Organic Fuji Apple, Seedless Red Grapes)",List(Banana),0.5138121546961326
"List(Organic Raspberries, Organic Hass Avocado)",List(Bag of Organic Bananas),0.5115273775216138
